# 7DaysOfCode
## Desafio de Preparação de Dados

### Objetivo
O objetivo deste desafio é preparar e limpar os dados do CEAPS (Cota para Exercício da Atividade Parlamentar dos Senadores) para análise. Este notebook abrange várias etapas de limpeza e transformação dos dados, incluindo correção de datas e tratamento de valores nulos.



### 1. Integração de Dados
- **Baixar os dados** do site: [Senado Transparência](https://www12.senado.leg.br/transparencia/dados-abertos-transparencia/dados-abertos-ceaps?utm_medium=email&_hsenc=p2ANqtz--WcZmbg4cw6KSG6BiOhIWO)

A seguir, carrego e integro os dados de despesas dos anos de 2019 a 2022.



In [1]:
import pandas as pd
# Carregando os dados de 2019 a 2022
ceaps_2019_2022 = pd.concat(
    pd.read_csv(
        f"data/despesa_ceaps_{ano}.csv",
        sep=';',
        decimal=',',
        encoding='ISO-8859-1',
        header=1
    )
    for ano in range(2019, 2023)
)


### 2. Entendendo as Bases
Explorar os dados para entender sua estrutura, identificando os primeiros e últimos valores, além de analisar as variáveis presentes.

Vou verificar a estrutura inicial do dataframe e identificar possíveis problemas.

In [2]:
# Verificar as primeiras e últimas linhas do dataframe
print(ceaps_2019_2022.head())
print(ceaps_2019_2022.tail())

# Informações gerais sobre o dataframe
ceaps_2019_2022.info()

# Resumo estatístico das variáveis numéricas
ceaps_2019_2022.describe()


    ANO  MES       SENADOR                                       TIPO_DESPESA  \
0  2019    1  ACIR GURGACZ  Aluguel de imóveis para escritório político, c...   
1  2019    1  ACIR GURGACZ  Aluguel de imóveis para escritório político, c...   
2  2019    1  ACIR GURGACZ  Aluguel de imóveis para escritório político, c...   
3  2019    1  ACIR GURGACZ  Aluguel de imóveis para escritório político, c...   
4  2019    2  ACIR GURGACZ  Aluguel de imóveis para escritório político, c...   

             CNPJ_CPF                     FORNECEDOR  DOCUMENTO        DATA  \
0  05.914.650/0001-66                       ENERGISA  006582758  04/01/2019   
1  05.914.650/0001-66                       ENERGISA  006582755  04/01/2019   
2      004.948.028-63  GILBERTO PISELO DO NASCIMENTO      00119  07/01/2019   
3  05.423.963/0001-11                  OI MÓVEL S.A.   86161151  25/12/2018   
4  05.914.650/0001-66                       ENERGISA  007236036  04/02/2019   

                                      

,ANO,MES,VALOR_REEMBOLSADO,COD_DOCUMENTO
count,69354.000000,69354.000000,69354.000000,6.935400e+04
mean,2020.415246,6.586412,1413.102937,2.153677e+06
std,1.162439,3.369575,3097.720012,2.258380e+04
min,2019.000000,1.000000,0.010000,2.112846e+06
25%,2019.000000,4.000000,158.205000,2.134379e+06
50%,2020.000000,7.000000,426.660000,2.153704e+06
75%,2021.000000,10.000000,1570.390000,2.172931e+06
max,2022.000000,12.000000,120000.000000,2.199784e+06


- verifica-se valores nulos nas variáveis de documento e detalhamento
- verifica-se que a data está no tipo errado


### 3. Tratamento e Limpeza dos Dados

#### 3.1 Correção de Datas
Identifiquei problemas nas datas que precisam ser corrigidos. Primeiro, vou converter a coluna de datas para o formato correto e identificar datas inválidas.

In [3]:
#Criando nova coluna de datas corretas para verificar se vai anular alguma data antes de atualizar a coluna original do dataset.
ceaps_2019_2022['DATA_CONVERTIDA'] = pd.to_datetime(ceaps_2019_2022['DATA'], format='%d/%m/%Y', errors='coerce')
print(ceaps_2019_2022['DATA_CONVERTIDA'].dtype)
print(ceaps_2019_2022['DATA_CONVERTIDA'].isnull().sum())

datetime64[ns]
3


In [4]:
#Identifiquei 3 linhas nulas
linhas_nulas = ceaps_2019_2022[ceaps_2019_2022['DATA_CONVERTIDA'].isnull()]
print(linhas_nulas[['DATA']] + " ano: " + linhas_nulas[['ANO']])

#Após a verificação reparei que as datas estão inconsistentes no campo de ano então vou alterar com seu correspondente da coluna ANO

       ANO DATA
10986  NaN  NaN
10362  NaN  NaN
2148   NaN  NaN


In [5]:

#Como são apenas três valores vou alterar no campo DATA da tabela original manualmente
ceaps_2019_2022.loc[10986, 'DATA'] = '31/01/2019'
ceaps_2019_2022.loc[10362, 'DATA'] = '05/05/2020'
ceaps_2019_2022.loc[2148, 'DATA'] = '29/10/2021'
#Vou reatibuir os valores novamente na coluna que criei aoós a alterar
ceaps_2019_2022['DATA_CONVERTIDA'] = pd.to_datetime(ceaps_2019_2022['DATA'], format='%d/%m/%Y', errors='coerce')
print(ceaps_2019_2022['DATA_CONVERTIDA'].dtype)
print(ceaps_2019_2022['DATA_CONVERTIDA'].isnull().sum())

datetime64[ns]
0


In [6]:
#Agora com a coluna data_convertida correta e verificada irei substituir os valores na data e excluir 
ceaps_2019_2022['DATA'] = ceaps_2019_2022['DATA_CONVERTIDA']
ceaps_2019_2022.drop(columns=['DATA_CONVERTIDA'], inplace=True)
ceaps_2019_2022.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69354 entries, 0 to 16802
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ANO                69354 non-null  int64         
 1   MES                69354 non-null  int64         
 2   SENADOR            69354 non-null  object        
 3   TIPO_DESPESA       69354 non-null  object        
 4   CNPJ_CPF           69354 non-null  object        
 5   FORNECEDOR         69354 non-null  object        
 6   DOCUMENTO          66391 non-null  object        
 7   DATA               69354 non-null  datetime64[ns]
 8   DETALHAMENTO       38583 non-null  object        
 9   VALOR_REEMBOLSADO  69354 non-null  float64       
 10  COD_DOCUMENTO      69354 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 8.4+ MB


#### 3.2 Análise de Anos Inesperados
Após corrigir as datas, vou verificar a presença de anos inesperados na coluna ano e na coluna data.

In [7]:
# Verificando anos válidos
import numpy as np
contagem_anos = ceaps_2019_2022['DATA'].dt.year.value_counts()
print(contagem_anos)

DATA
2019    21576
2021    16844
2022    16751
2020    14003
2018       88
2023       75
2002        8
2010        4
2014        2
2009        1
2016        1
2000        1
Name: count, dtype: int64


In [179]:
print(ceaps_2019_2022['ANO'].value_counts())
#verifica-se que não tem valor incorreto nos anos então posso arrumar os anos inesperados da data com base na coluna ano


ANO
2019    21634
2021    16827
2022    16803
2020    14090
Name: count, dtype: int64


In [180]:
#Array com os anos inesperados no campo data
anos_erro = [2000,2002,2016,2009,2010,2014,2023,2018]
#Criando nova tabela com as datas com erro pra reatribuir as datas corretas no dataset
linhas_erro = ceaps_2019_2022[ceaps_2019_2022['DATA'].dt.year.isin(anos_erro)]
def corrigir_ano(row):
    if row['DATA'].year in anos_erro:
        return row['DATA'].replace(year = row['ANO'])
linhas_erro['DATA'] = linhas_erro.apply(lambda row: corrigir_ano(row), axis=1)
print(linhas_erro['DATA'].dt.year.value_counts())
ceaps_2019_2022.loc[ceaps_2019_2022['DATA'].dt.year.isin(anos_erro), 'DATA'] = linhas_erro['DATA'].values
print(ceaps_2019_2022['DATA'].dt.year.value_counts())

DATA
2019    94
2022    82
2020     2
2021     2
Name: count, dtype: int64
DATA
2019    21670
2021    16846
2022    16833
2020    14005
Name: count, dtype: int64


C:\Users\LisLima\AppData\Local\Temp\ipykernel_19512\2423477421.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linhas_erro['DATA'] = linhas_erro.apply(lambda row: corrigir_ano(row), axis=1)


#### 3.3 Tratamento de Documentos Nulos
Vamos preencher os valores nulos na coluna 'DOCUMENTO' com base no documento mais comum para cada senador.


In [181]:
#Irei seguir a analise tratando os documentos nulos 
documentos_nulos = ceaps_2019_2022[ceaps_2019_2022['DOCUMENTO'].isnull()]
print(documentos_nulos['SENADOR'].value_counts())
ceaps_2019_2022['SENADOR'].unique()
print(ceaps_2019_2022['SENADOR'].value_counts())



SENADOR
FABIANO CONTARATO          563
JOSÉ SERRA                 262
HUMBERTO COSTA             250
JAQUES WAGNER              167
MARIA DO CARMO ALVES       154
RENAN CALHEIROS            119
FLÁVIO BOLSONARO           119
ANTONIO ANASTASIA          118
EDUARDO GOMES              108
FERNANDO BEZERRA COELHO    106
ROGÉRIO CARVALHO            96
CARLOS VIANA                90
SIMONE TEBET                84
ZENAIDE MAIA                66
JARBAS VASCONCELOS          58
ROBERTO ROCHA               53
PLÍNIO VALÉRIO              48
CARLOS FÁVARO               45
KÁTIA ABREU                 43
RODRIGO CUNHA               41
JUÍZA SELMA                 41
EDUARDO BRAGA               35
DAVI ALCOLUMBRE             33
ORIOVISTO GUIMARÃES         24
GIORDANO                    23
MAILZA GOMES                21
STYVENSON VALENTIM          20
RANDOLFE RODRIGUES          19
JORGINHO MELLO              18
VANDERLAN CARDOSO           18
ALEXANDRE SILVEIRA          15
LEILA BARROS                12


In [182]:
# Criei um dicionário com o documento mais comum para cada senador
documento_por_senador = ceaps_2019_2022.groupby('SENADOR')['DOCUMENTO'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None).to_dict()

def preencher_documento(row):
    if pd.isnull(row['DOCUMENTO']):
        return documento_por_senador.get(row['SENADOR'])
    return row['DOCUMENTO']

ceaps_2019_2022['DOCUMENTO'] = ceaps_2019_2022.apply(preencher_documento, axis=1)
# Verificar se há mais nulos, no caso o da leila barros está nulo mas ela nunca preencheu 
remaining_nulls = ceaps_2019_2022[ceaps_2019_2022['DOCUMENTO'].isnull()]
print(remaining_nulls['SENADOR'].value_counts())
ceaps_2019_2022.info()
#Quanto aos detalhes nulos é muito pessoal e específico dispensa preenchimento dos campos nulos

SENADOR
LEILA BARROS    12
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 69354 entries, 0 to 16802
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ANO                69354 non-null  int64         
 1   MES                69354 non-null  int64         
 2   SENADOR            69354 non-null  object        
 3   TIPO_DESPESA       69354 non-null  object        
 4   CNPJ_CPF           69354 non-null  object        
 5   FORNECEDOR         69354 non-null  object        
 6   DOCUMENTO          69342 non-null  object        
 7   DATA               69354 non-null  datetime64[ns]
 8   DETALHAMENTO       38583 non-null  object        
 9   VALOR_REEMBOLSADO  69354 non-null  float64       
 10  COD_DOCUMENTO      69354 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 8.4+ MB


In [183]:
# vou seguir a análise para ver se encontro mais valores estranhos
valores_unicos = {coluna: ceaps_2019_2022[coluna].unique() for coluna in ceaps_2019_2022.columns}
for coluna, valores in valores_unicos.items():
    print(f"Valores únicos na coluna '{coluna}':")
    print(valores)
    print()
#Verifica-se que aparentemente os valores estão corretos

Valores únicos na coluna 'ANO':
[2019 2020 2021 2022]

Valores únicos na coluna 'MES':
[ 1  2  3  4  5  6  7  8  9 10 11 12]

Valores únicos na coluna 'SENADOR':
['ACIR GURGACZ' 'AÉCIO NEVES' 'ALESSANDRO VIEIRA' 'ALOYSIO NUNES FERREIRA'
 'ALVARO DIAS' 'ANA AMÉLIA' 'ÂNGELA PORTELA' 'ANGELO CORONEL'
 'ANTONIO ANASTASIA' 'ANTÔNIO CARLOS VALADARES' 'ARMANDO MONTEIRO'
 'AROLDE DE OLIVEIRA' 'ATAÍDES OLIVEIRA' 'BENEDITO DE LIRA' 'BLAIRO MAGGI'
 'CARLOS VIANA' 'CÁSSIO CUNHA LIMA' 'CHICO RODRIGUES' 'CID GOMES'
 'CIRO NOGUEIRA' 'CONFÚCIO MOURA' 'DALIRIO BEBER' 'DANIELLA RIBEIRO'
 'DÁRIO BERGER' 'DAVI ALCOLUMBRE' 'EDISON LOBÃO' 'EDUARDO AMORIM'
 'EDUARDO BRAGA' 'EDUARDO GIRÃO' 'EDUARDO GOMES' 'ELIZIANE GAMA'
 'ELMANO FÉRRER' 'ESPERIDIÃO AMIN' 'FABIANO CONTARATO'
 'FERNANDO BEZERRA COELHO' 'FERNANDO COLLOR' 'FLÁVIO ARNS'
 'FLÁVIO BOLSONARO' 'FLEXA RIBEIRO' 'GARIBALDI ALVES FILHO'
 'GLEISI HOFFMANN' 'GUARACY SILVEIRA' 'HÉLIO JOSÉ' 'HUMBERTO COSTA'
 'IRAJÁ' 'IVO CASSOL' 'IZALCI LUCAS' 'JADER BARBALH


### 5. Salvando o DataFrame Tratado

Após limpar e preparar os dados, vamos salvar o DataFrame resultante em diferentes formatos para uso futuro.

In [184]:
ceaps_2019_2022.to_csv('ceaps_2019_2022_tratado.csv', index=False, sep=';', decimal=',', encoding='ISO-8859-1')


### 4. Conclusões
Após a limpeza e preparação dos dados, corrigimos datas inválidas, tratamos anos inesperados, e preenchemos documentos nulos. Estes passos são essenciais para garantir a integridade e a precisão das análises subsequentes.